![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)



[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/PUBLIC_HEALTH_MENTIONS_DE.ipynb)

## **Classify Public Health Mentions in German**

## Colab Setup

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [3]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.2.8
Spark NLP_JSL Version : 4.2.8


In [4]:
text_list = ["""Die Schmerzen im Oberschenkel sind dann aber shcnell besser geworden, vielleicht war es auhc nur der Schrecke nach dem Stich.""",
             """Der roman handelt von einer mutter die ihre tochter sucht nachdem diese im urlaub verschwunden ist.""",
             """Der Orthopäde/ Chirurg bei dem ich zuerst war, wollte nicht operieren- er meinte die Probelmatik würde so nicht gelöst.""",
             """Wir waren bei einem Wine Tasting in der Steiermark.""",
             """Auch der Betriebsarzt ist zu der Diagnose gekommen und sich weitgehend sicher, dass ich Zuckerprobleme habe.""",  
             """Es dürfte dafür aber keine relativ einfache Lösung geben - man muss wohl nur die Lüftung austauschen."""]

## **bert_sequence_classifier_health_mentions_bert**

In [5]:
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

sequenceClassifier = MedicalBertForSequenceClassification.pretrained("bert_sequence_classifier_health_mentions_bert", "de", "clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("class_")

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    sequenceClassifier    
])


bert_sequence_classifier_health_mentions_bert download started this may take some time.
[OK!]


In [6]:
files = [i for i in (range(1, len(text_list)+1))]

df = spark.createDataFrame(pd.DataFrame({'index' : files,'text': text_list}))

In [7]:
result = pipeline.fit(df).transform(df)

In [8]:
result.select('index',F.explode(F.arrays_zip(result.document.result, 
                                             result.class_.result)).alias("col"))\
      .select('index',F.expr("col['0']").alias("sentence"),
                      F.expr("col['1']").alias("prediction")).show(truncate=100)

+-----+----------------------------------------------------------------------------------------------------+--------------+
|index|                                                                                            sentence|    prediction|
+-----+----------------------------------------------------------------------------------------------------+--------------+
|    1|Die Schmerzen im Oberschenkel sind dann aber shcnell besser geworden, vielleicht war es auhc nur ...|health-related|
|    2| Der roman handelt von einer mutter die ihre tochter sucht nachdem diese im urlaub verschwunden ist.|    non-health|
|    3|Der Orthopäde/ Chirurg bei dem ich zuerst war, wollte nicht operieren- er meinte die Probelmatik ...|health-related|
|    4|                                                 Wir waren bei einem Wine Tasting in der Steiermark.|    non-health|
|    5|Auch der Betriebsarzt ist zu der Diagnose gekommen und sich weitgehend sicher, dass ich Zuckerpro...|health-related|
|    6|E

## **bert_sequence_classifier_health_mentions_gbert**

In [9]:
sequenceClassifier = MedicalBertForSequenceClassification.pretrained("bert_sequence_classifier_health_mentions_gbert", "de", "clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("class_")

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    sequenceClassifier    
])

bert_sequence_classifier_health_mentions_gbert download started this may take some time.
[OK!]


In [10]:
files = [i for i in (range(1, len(text_list)+1))]

df = spark.createDataFrame(pd.DataFrame({'index' : files,'text': text_list}))

In [11]:
result = pipeline.fit(df).transform(df)

In [12]:
result.select('index',F.explode(F.arrays_zip(result.document.result, 
                                             result.class_.result)).alias("col"))\
      .select('index',F.expr("col['0']").alias("sentence"),
                      F.expr("col['1']").alias("prediction")).show(truncate=100)

+-----+----------------------------------------------------------------------------------------------------+--------------+
|index|                                                                                            sentence|    prediction|
+-----+----------------------------------------------------------------------------------------------------+--------------+
|    1|Die Schmerzen im Oberschenkel sind dann aber shcnell besser geworden, vielleicht war es auhc nur ...|health-related|
|    2| Der roman handelt von einer mutter die ihre tochter sucht nachdem diese im urlaub verschwunden ist.|    non-health|
|    3|Der Orthopäde/ Chirurg bei dem ich zuerst war, wollte nicht operieren- er meinte die Probelmatik ...|health-related|
|    4|                                                 Wir waren bei einem Wine Tasting in der Steiermark.|    non-health|
|    5|Auch der Betriebsarzt ist zu der Diagnose gekommen und sich weitgehend sicher, dass ich Zuckerpro...|health-related|
|    6|E

## **bert_sequence_classifier_health_mentions_gbert_large**

In [13]:
sequenceClassifier = MedicalBertForSequenceClassification.pretrained("bert_sequence_classifier_health_mentions_gbert_large", "de", "clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("class_")

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    sequenceClassifier    
])

bert_sequence_classifier_health_mentions_gbert_large download started this may take some time.
[OK!]


In [14]:
files = [i for i in (range(1, len(text_list)+1))]

df = spark.createDataFrame(pd.DataFrame({'index' : files,'text': text_list}))

In [15]:
result = pipeline.fit(df).transform(df)

In [16]:
result.select('index',F.explode(F.arrays_zip(result.document.result, 
                                             result.class_.result)).alias("col"))\
      .select('index',F.expr("col['0']").alias("sentence"),
                      F.expr("col['1']").alias("prediction")).show(truncate=100)

+-----+----------------------------------------------------------------------------------------------------+--------------+
|index|                                                                                            sentence|    prediction|
+-----+----------------------------------------------------------------------------------------------------+--------------+
|    1|Die Schmerzen im Oberschenkel sind dann aber shcnell besser geworden, vielleicht war es auhc nur ...|health-related|
|    2| Der roman handelt von einer mutter die ihre tochter sucht nachdem diese im urlaub verschwunden ist.|    non-health|
|    3|Der Orthopäde/ Chirurg bei dem ich zuerst war, wollte nicht operieren- er meinte die Probelmatik ...|health-related|
|    4|                                                 Wir waren bei einem Wine Tasting in der Steiermark.|    non-health|
|    5|Auch der Betriebsarzt ist zu der Diagnose gekommen und sich weitgehend sicher, dass ich Zuckerpro...|health-related|
|    6|E

## **bert_sequence_classifier_health_mentions_medbert**

In [17]:
sequenceClassifier = MedicalBertForSequenceClassification.pretrained("bert_sequence_classifier_health_mentions_medbert", "de", "clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("class_")

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    sequenceClassifier    
])

bert_sequence_classifier_health_mentions_medbert download started this may take some time.
[OK!]


In [18]:
files = [i for i in (range(1, len(text_list)+1))]

df = spark.createDataFrame(pd.DataFrame({'index' : files,'text': text_list}))

In [19]:
result = pipeline.fit(df).transform(df)

In [20]:
result.select('index',F.explode(F.arrays_zip(result.document.result, 
                                             result.class_.result)).alias("col"))\
      .select('index',F.expr("col['0']").alias("sentence"),
                      F.expr("col['1']").alias("prediction")).show(truncate=100)

+-----+----------------------------------------------------------------------------------------------------+--------------+
|index|                                                                                            sentence|    prediction|
+-----+----------------------------------------------------------------------------------------------------+--------------+
|    1|Die Schmerzen im Oberschenkel sind dann aber shcnell besser geworden, vielleicht war es auhc nur ...|health-related|
|    2| Der roman handelt von einer mutter die ihre tochter sucht nachdem diese im urlaub verschwunden ist.|    non-health|
|    3|Der Orthopäde/ Chirurg bei dem ich zuerst war, wollte nicht operieren- er meinte die Probelmatik ...|health-related|
|    4|                                                 Wir waren bei einem Wine Tasting in der Steiermark.|    non-health|
|    5|Auch der Betriebsarzt ist zu der Diagnose gekommen und sich weitgehend sicher, dass ich Zuckerpro...|health-related|
|    6|E